In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/app')

In [3]:
import os
API_URL = os.getenv('API_URL')

In [10]:
from api_client import APIClient

client = APIClient()

await client.set_email("2280905@gmail.com")

await client.get_meetings()


{'total': 764,
 'meetings': [{'meeting_id': '6a3451aa-f18c-4211-8140-fbc77ea7ba05',
   'timestamp': '2024-12-13T11:30:24',
   'is_indexed': True,
   'access_level': 'owner',
   'is_owner': True,
   'speakers': ['Dmitry Grankin', 'Viktoria Kharlamova']},
  {'meeting_id': '77af4031-a5a9-4dae-ad90-80d12bdadde0',
   'timestamp': '2024-12-12T16:00:31',
   'is_indexed': True,
   'access_level': 'owner',
   'is_owner': True,
   'speakers': ['AI Marketing Directory', 'Dmitry Grankin']},
  {'meeting_id': '860f9495-13a2-486e-8abf-3b5a3ec22a48',
   'timestamp': '2024-12-12T09:59:27',
   'is_indexed': True,
   'access_level': 'owner',
   'is_owner': True,
   'speakers': ['Dmitry Grankin', 'Евгения Садовская', 'Елизавета Иванова']},
  {'meeting_id': '4f64ba1e-fd80-4b5d-a1be-345225028f6c',
   'timestamp': '2024-12-11T18:30:54',
   'is_indexed': True,
   'access_level': 'owner',
   'is_owner': True,
   'speakers': ['Dmitriy Grankin', 'Lara Vargas']},
  {'meeting_id': '2a92e93e-dc1c-455d-acbb-18255519

In [8]:
from IPython.display import clear_output, display, Markdown
from typing import Optional, List
from functools import wraps
from helpers import jupyter_stream_output

@jupyter_stream_output(markdown=True)
async def stream_chat(
    client,
    query: str,
    meeting_ids: Optional[List[str]] = None,
    speakers: Optional[List[str]] = None,
    thread_id: Optional[str] = None,
    model: str = "gpt-4o-mini",
    temperature: float = 0.7
):
    response = client.chat(
        query=query,
        meeting_ids=meeting_ids,
        speakers=speakers,
        thread_id=thread_id,
        model=model,
        temperature=temperature
    )
    yield response

In [11]:

await stream_chat(
    client=client,
    query="What was discussed in the meeting?"
)

To provide you with a summary of what was discussed in the meetings, I can reference specific meetings based on your interests and previous discussions. Here are some highlights from recent meetings:

1. **Meeting 20 - December 06, 2024**: You shared your experiences at an offline business club event, where you recorded presentations and discussed the importance of creating value through shared content and collaborative indexing in online events. You mentioned, "And what I did is that I recorded some... presentations and summarized them" [20].

2. **Meeting 3 - December 06, 2024**: You discussed the features of a transcription and summarization tool used during meetings, emphasizing its ability to provide real-time transcripts and summaries. You noted, "it delivers the thin real transcript and it pretifies the transcript that highlights important words" [3].

3. **Meeting 21 - December 05, 2024**: You and Alex Shevliakov talked about the development of a platform that includes features

{'type': 'final',
 'thread_id': '02d08a42-4ae3-4443-a7a9-b55c7a1e2d94',
 'output': 'To provide you with a summary of what was discussed in the meetings, I can reference specific meetings based on your interests and previous discussions. Here are some highlights from recent meetings:\n\n1. **[Meeting 2](/meeting/4b253b0e-6b15-452a-b66c-da5f06ec3fcf)0 - December 06, 2024**: You shared your experiences at an offline business club event, where you recorded presentations and discussed the importance of creating value through shared content and collaborative indexing in online events. You mentioned, "And what I did is that I recorded some... presentations and summarized them" [20](/meeting/15af0212-e418-4476-8586-085581bfd8c9).\n\n2. **[Meeting 3](/meeting/161f80a4-135f-4aa4-b2fc-84dec9d6445c) - December 06, 2024**: You discussed the features of a transcription and summarization tool used during meetings, emphasizing its ability to provide real-time transcripts and summaries. You noted, "it 